In [ ]:
import zipfile
 
with zipfile.ZipFile('/home/jupyter/datasphere/project/archive_food.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [12]:
%pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [13]:
import torch
print(torch.cuda.is_available())

True


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet101
from PIL import Image
import os
import pandas as pd

# Create the dataset class
class Food101(Dataset):
    def __init__(self, dataframe, base_dir, transform=None):
        self.dataframe = dataframe
        self.base_dir = base_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_dir, self.dataframe.path.iloc[idx])
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.label.iloc[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

images_dir = 'archive_food/images/'

dataframe = []
for root, _, files in os.walk(images_dir):
    for file in files:
        if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
            file_path = os.path.join(root, file)
            label = os.path.basename(root)
            formatted_path = os.path.join(label, file)
            dataframe.append({'path': formatted_path, 'label': label})

dataframe = pd.DataFrame(dataframe)

transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
food_dataset = Food101(dataframe, images_dir, transform=transform)

train_size = int(0.7 * len(food_dataset))
val_size = int(0.15 * len(food_dataset))
test_size = len(food_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(food_dataset, [train_size, val_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the model
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.resnet101 = resnet101(pretrained=True)
        for param in self.resnet101.parameters():
            param.requires_grad = False
        self.resnet101.fc = nn.Linear(self.resnet101.fc.in_features, num_classes)

    def forward(self, x):
        x = self.resnet101(x)
        return x

model = ResNetClassifier(num_classes=len(food_dataset.dataframe['label'].unique())).cuda()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
print(dataframe)

                           path         label
0         foie_gras/1244707.jpg     foie_gras
1         foie_gras/1335868.jpg     foie_gras
2         foie_gras/1176321.jpg     foie_gras
3         foie_gras/1421924.jpg     foie_gras
4         foie_gras/1051563.jpg     foie_gras
...                         ...           ...
20195  caesar_salad/1397383.jpg  caesar_salad
20196   caesar_salad/126205.jpg  caesar_salad
20197  caesar_salad/1517451.jpg  caesar_salad
20198    caesar_salad/15081.jpg  caesar_salad
20199  caesar_salad/1351688.jpg  caesar_salad

[20200 rows x 2 columns]


In [29]:

import torch
import torch.nn.functional as F
import torch.optim as optim

class_names = sorted(set(target for batch in train_loader for target in batch[1]))
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

def test(model, device, test_loader, class_to_idx):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = torch.tensor([class_to_idx[t] for t in target], dtype=torch.long).to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            print(f'Test set: Batch {batch_idx+1} of {len(test_loader)} - Loss: {test_loss/(batch_idx+1):.4f}, '
                  f'Accuracy: {correct}/{len(test_loader.dataset)} '
                  f'({100. * correct / len(test_loader.dataset):.0f}%)')

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

def validate(model, device, val_loader, class_to_idx):
    model.eval()
    total_correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            data = data.to(device)
            target = torch.tensor([class_to_idx[t] for t in target], dtype=torch.long).to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == target).sum().item()

    accuracy = total_correct / len(val_loader.dataset)
    print(f'Validation Accuracy: {accuracy:.4f}')

def train(model, device, train_loader, optimizer, epoch, class_to_idx):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = torch.tensor([class_to_idx[t] for t in target], dtype=torch.long).to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(25):
    train(model, device, train_loader, optimizer, epoch, class_to_idx)
    validate(model, device, val_loader, class_to_idx)
    torch.save(model.state_dict(), 'lastepoch_model.pth')

test(model, device, test_loader, class_to_idx)
torch.save(model.state_dict(), 'tested_model.pth')

Train Epoch: 0 [0/14140 (0%)]	Loss: 4.673238
Train Epoch: 0 [640/14140 (5%)]	Loss: 4.556993
Train Epoch: 0 [1280/14140 (9%)]	Loss: 4.447625
Train Epoch: 0 [1920/14140 (14%)]	Loss: 3.972295
Train Epoch: 0 [2560/14140 (18%)]	Loss: 4.136531
Train Epoch: 0 [3200/14140 (23%)]	Loss: 3.443301
Train Epoch: 0 [3840/14140 (27%)]	Loss: 3.513201
Train Epoch: 0 [4480/14140 (32%)]	Loss: 3.432453
Train Epoch: 0 [5120/14140 (36%)]	Loss: 3.115723
Train Epoch: 0 [5760/14140 (41%)]	Loss: 3.122023
Train Epoch: 0 [6400/14140 (45%)]	Loss: 3.158931
Train Epoch: 0 [7040/14140 (50%)]	Loss: 2.724830
Train Epoch: 0 [7680/14140 (54%)]	Loss: 3.081381
Train Epoch: 0 [8320/14140 (59%)]	Loss: 2.841461
Train Epoch: 0 [8960/14140 (63%)]	Loss: 2.470081
Train Epoch: 0 [9600/14140 (68%)]	Loss: 2.986862
Train Epoch: 0 [10240/14140 (72%)]	Loss: 2.714617
Train Epoch: 0 [10880/14140 (77%)]	Loss: 2.843295
Train Epoch: 0 [11520/14140 (81%)]	Loss: 2.484378
Train Epoch: 0 [12160/14140 (86%)]	Loss: 2.367414
Train Epoch: 0 [12800/1